<a href="https://colab.research.google.com/github/Vishal8500/Parkinson-Disease-Prediction/blob/main/Speech_parkinson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import os

# Paths to your folders
parkinsons_path = "/content/dataset/Parkinson"
healthy_path = "/content/dataset/Heathly"

# Helper function to load, normalize, and resample
def process_audio(file_path, target_sr=22050):
    y, sr = librosa.load(file_path, sr=None)  # Original sampling rate
    y = y / np.max(np.abs(y))  # Normalize to [-1, 1]
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
    return y, target_sr

# Create dataset lists
audio_data = []
labels = []

# Load Parkinson's patients (label 1)
for file in os.listdir(parkinsons_path):
    if file.endswith('.wav'):
        path = os.path.join(parkinsons_path, file)
        y, sr = process_audio(path)
        audio_data.append(y)
        labels.append(1)

# Load Healthy persons (label 0)
for file in os.listdir(healthy_path):
    if file.endswith('.wav'):
        path = os.path.join(healthy_path, file)
        y, sr = process_audio(path)
        audio_data.append(y)
        labels.append(0)

print(f"Total samples: {len(audio_data)} (Parkinson's: {labels.count(1)}, Healthy: {labels.count(0)})")


Total samples: 81 (Parkinson's: 40, Healthy: 41)


In [4]:
!pip install noisereduce

In [5]:
import librosa
import numpy as np
import os
import matplotlib.pyplot as plt
import noisereduce as nr
import librosa.display

# Function: Preprocess, Frame, Apply Window, and STFT
def preprocess_and_convert_to_mel(file_path, target_sr=22050, frame_length=2048, hop_length=512, n_mels=128, target_length=66150):
    # 1. Load Audio
    y, sr = librosa.load(file_path, sr=None)

    # 2. Resample (if needed)
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

    # 3. Normalize to [-1, 1]
    y = y / np.max(np.abs(y))

    # 4. Trim Silence (30 dB threshold)
    y, _ = librosa.effects.trim(y, top_db=30)

    # 5. Noise Reduction (Optional)
    y = nr.reduce_noise(y=y, sr=target_sr)

    # 6. Pad/Truncate to Fixed Length
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)))
    else:
        y = y[:target_length]

    # 7. Framing: Split audio into overlapping frames
    frames = librosa.util.frame(y, frame_length=frame_length, hop_length=hop_length).T

    # 8. Windowing: Apply a Hamming window to each frame
    window = np.hamming(frame_length)
    windowed_frames = frames * window

    # 9. Apply STFT to convert to the frequency domain
    stft = librosa.stft(y, n_fft=frame_length, hop_length=hop_length, window='hamming')

    # 10. Convert to Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(S=np.abs(stft)**2, sr=target_sr, n_mels=n_mels)

    # 11. Convert Power to dB Scale
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_spec_db

In [9]:
import os
import numpy as np

def process_dataset(parkinsons_path, healthy_path, output_dir):
    # Create subfolders for each class
    parkinsons_folder = os.path.join(output_dir, "parkinson")
    healthy_folder = os.path.join(output_dir, "healthy")
    os.makedirs(parkinsons_folder, exist_ok=True)
    os.makedirs(healthy_folder, exist_ok=True)

    labels = []
    all_mels = []

    # Use a list of tuples to iterate
    for label, (class_path, output_folder) in enumerate(
            [(parkinsons_path, parkinsons_folder), (healthy_path, healthy_folder)]):
        for file in os.listdir(class_path):
            if file.endswith('.wav'):
                file_path = os.path.join(class_path, file)

                # Preprocess and convert to Mel Spectrogram
                mel_spec = preprocess_and_convert_to_mel(file_path)

                # Save the Mel spectrogram as a .npy file in the respective class folder
                output_file = os.path.join(output_folder, f"{file.replace('.wav', '.npy')}")
                np.save(output_file, mel_spec)

                all_mels.append(output_file)
                labels.append(label)

    print(f"✅ Processed {len(all_mels)} samples (Parkinson's: {labels.count(0)}, Healthy: {labels.count(1)})")
    return all_mels, labels

# Paths to Dataset
parkinsons_path = "/content/dataset/Parkinson"
healthy_path = "/content/dataset/Heathly"
output_dir = "/content/mel_spectrograms/"

# Process Dataset and Save Mel Spectrograms
mels, labels = process_dataset(parkinsons_path, healthy_path, output_dir)


✅ Processed 81 samples (Parkinson's: 40, Healthy: 41)


In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt

def save_mel_as_image(npy_file, image_path):
    # Load the Mel spectrogram from the .npy file
    mel_spec = np.load(npy_file)

    # Create the image and save it
    plt.figure(figsize=(mel_spec.shape[1] / 100, mel_spec.shape[0] / 100), dpi=100)
    plt.imshow(mel_spec, cmap='inferno', aspect='auto', origin='lower')
    plt.axis('off')  # Hide axes for cleaner image
    plt.tight_layout()

    # Save the image to the specified path
    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)

    # Close the plot to avoid memory issues
    plt.close()

def convert_npy_to_images(mel_dir, output_image_dir):
    # Iterate through each class (parkinson and healthy)
    for class_folder in os.listdir(mel_dir):
        class_folder_path = os.path.join(mel_dir, class_folder)

        # Check if it is a directory (parkinson or healthy)
        if os.path.isdir(class_folder_path):
            # Create the corresponding class folder in the output directory
            class_output_folder = os.path.join(output_image_dir, class_folder)
            os.makedirs(class_output_folder, exist_ok=True)

            # Process each .npy file in the class folder
            for file in os.listdir(class_folder_path):
                if file.endswith('.npy'):
                    npy_file_path = os.path.join(class_folder_path, file)

                    # Define the output image path
                    image_path = os.path.join(class_output_folder, file.replace('.npy', '.png'))

                    # Save the Mel spectrogram as an image
                    save_mel_as_image(npy_file_path, image_path)

    print(f"✅ Converted Mel spectrograms to images in {output_image_dir}")

# Define the directory with Mel spectrogram .npy files
mel_spectrogram_dir = "/content/mel_spectrograms/"
output_image_dir = "/content/mel_spectrogram_images/"

# Convert Mel spectrograms to images and preserve folder structure
convert_npy_to_images(mel_spectrogram_dir, output_image_dir)


✅ Converted Mel spectrograms to images in /content/mel_spectrogram_images/


In [21]:
import os
from PIL import Image

def resize_and_replace_images(image_dir, target_size=(224, 224)):
    # Iterate through each image in the directory
    for class_folder in os.listdir(image_dir):
        class_folder_path = os.path.join(image_dir, class_folder)

        # Check if it is a directory (parkinson or healthy)
        if os.path.isdir(class_folder_path):
            # Process each image in the class folder
            for file in os.listdir(class_folder_path):
                if file.endswith('.png'):
                    image_path = os.path.join(class_folder_path, file)

                    # Open the image using PIL
                    img = Image.open(image_path)

                    # Resize the image
                    img_resized = img.resize(target_size)

                    # Save the resized image, replacing the original one
                    img_resized.save(image_path)

    print(f"✅ Resized and replaced images in {image_dir}")

# Define the directory with Mel spectrogram images
mel_spectrogram_image_dir = "/content/mel_spectrogram_images/"

# Resize and replace original images with the new ones
resize_and_replace_images(mel_spectrogram_image_dir)


✅ Resized and replaced images in /content/mel_spectrogram_images/


In [22]:
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt

# Set up the image size for resizing
target_size = (224, 224)

# Define paths
image_dir = "/content/mel_spectrogram_images/"  # Change this to your mel spectrogram images directory
augmented_dir = "/content/mel_spectrogram_images/augmented"  # Augmented images will be saved here

# Set up ImageDataGenerator with augmentations
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotate images within 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by 20%
    height_shift_range=0.2,  # Shift images vertically by 20%
    shear_range=0.2,  # Apply shearing transformations
    zoom_range=0.2,  # Zoom in or out on images by 20%
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill pixels that were transformed
)

# Function to apply augmentation and save images
def augment_and_save_images(image_dir):
    for class_folder in os.listdir(image_dir):
        class_folder_path = os.path.join(image_dir, class_folder)

        # Check if it is a directory (parkinson or healthy)
        if os.path.isdir(class_folder_path):
            # Create the corresponding folder in the augmented directory
            class_output_folder = os.path.join(augmented_dir, class_folder)
            os.makedirs(class_output_folder, exist_ok=True)

            # Process each .png file in the class folder
            for file in os.listdir(class_folder_path):
                if file.endswith('.png'):  # Process only .png images
                    image_path = os.path.join(class_folder_path, file)

                    # Load the image
                    img = load_img(image_path)
                    x = img_to_array(img)
                    x = x.reshape((1,) + x.shape)  # Reshape for augmentation

                    # Generate batches of augmented images and save them
                    i = 0
                    for batch in datagen.flow(x, batch_size=1, save_to_dir=class_output_folder, save_prefix='aug', save_format='png'):
                        i += 1
                        if i > 10:  # Generate 10 augmented images per original image
                            break

    print("✅ Augmentation and saving completed!")

# Function to oversample the minority class
def oversample_class(class_folder_path, target_count):
    # Get list of image files in the class folder
    files = [f for f in os.listdir(class_folder_path) if f.endswith('.png')]

    # Calculate how many samples are needed to reach the target count
    while len(files) < target_count:
        # Randomly select a file to duplicate
        file_to_duplicate = random.choice(files)

        # Create a copy of the selected file in the same folder
        shutil.copy(os.path.join(class_folder_path, file_to_duplicate), os.path.join(class_folder_path, f"dup_{len(files)}.png"))
        files.append(f"dup_{len(files)}.png")

    print(f"✅ Oversampled {class_folder_path} to {target_count} images")

# Run the augmentation
augment_and_save_images(image_dir)

# Example: Oversample the Parkinson's class to 200 images
parkinson_folder = os.path.join(image_dir, 'parkinson')
oversample_class(parkinson_folder, target_count=200)

# Example: Oversample the Healthy class to 200 images
healthy_folder = os.path.join(image_dir, 'healthy')
oversample_class(healthy_folder, target_count=200)


✅ Augmentation and saving completed!
✅ Oversampled /content/mel_spectrogram_images/parkinson to 200 images
✅ Oversampled /content/mel_spectrogram_images/healthy to 200 images


In [30]:
import os
import shutil
import random

def split_dataset(source_dir, output_dir, split_ratio=0.9):
    # Create the directories for train and validation
    train_dir = os.path.join(output_dir, 'train')
    valid_dir = os.path.join(output_dir, 'validation')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

    # Loop through each class (parkinson and healthy)
    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)

        # Check if it's a directory (parkinson or healthy)
        if os.path.isdir(class_path):
            # Create subdirectories for train and validation within each class folder
            train_class_dir = os.path.join(train_dir, class_name)
            valid_class_dir = os.path.join(valid_dir, class_name)
            os.makedirs(train_class_dir, exist_ok=True)
            os.makedirs(valid_class_dir, exist_ok=True)

            # Get all .png files for the current class
            files = [f for f in os.listdir(class_path) if f.endswith('.png')]

            # Shuffle the files to ensure random splitting
            random.shuffle(files)

            # Calculate the number of files for the training set
            num_train = int(len(files) * split_ratio)

            # Move files to the corresponding directories (train/validation)
            for i, file in enumerate(files):
                src = os.path.join(class_path, file)

                if i < num_train:
                    dest = os.path.join(train_class_dir, file)
                else:
                    dest = os.path.join(valid_class_dir, file)

                # Copy the file to the destination
                shutil.copy2(src, dest)

    print(f"✅ Dataset split completed. Training set: {len(os.listdir(train_class_dir))} files, Validation set: {len(os.listdir(valid_class_dir))} files")

# Define the source directory and the output directory for the split
source_dir = '/content/mel_spectrogram_images/augmented'
output_dir = '/content/split_mel_spectrograms'

# Split the dataset into train and validation sets (90-10 split)
split_dataset(source_dir, output_dir, split_ratio=0.9)


✅ Dataset split completed. Training set: 386 files, Validation set: 43 files


In [28]:
import os
import shutil

# Prompt user for folder path
folder_path = input("Enter the folder path to delete all files: ")

# Check if folder exists
if os.path.exists(folder_path):
    # Delete all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Delete file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder and contents
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
    print("All files deleted successfully.")
else:
    print("Folder not found!")


Enter the folder path to delete all files: /content/split_mel_spectrograms
Deleted: /content/split_mel_spectrograms/validation
Deleted: /content/split_mel_spectrograms/train
All files deleted successfully.


In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Load ResNet50 model with pre-trained weights and exclude the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers except the top layers
base_model.trainable = False  # Freeze all layers in the base model initially

# Add custom layers on top of ResNet50
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Add Dropout for regularization
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model (use a low learning rate for fine-tuning)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the frozen base model
history = model.fit(train_generator, epochs=15, steps_per_epoch=100)

Epoch 1/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 29s 149ms/step - accuracy: 0.4971 - loss: 1.3298
Epoch 2/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.4817 - loss: 0.9366 
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 108ms/step - accuracy: 0.4767 - loss: 0.7243
Epoch 4/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/step - accuracy: 0.4813 - loss: 0.7007
Epoch 5/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 124ms/step - accuracy: 0.4914 - loss: 0.6932
Epoch 6/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.4720 - loss: 0.6932
Epoch 7/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 0.5091 - loss: 0.6932
Epoch 8/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 111ms/step - accuracy: 0.5115 - loss: 0.6931
Epoch 9/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.5049 - loss: 0.6931
Epoch 10/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 21s 111ms/step - accuracy: 0.5185 - loss: 0.6931
Epoch 11/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 110ms/step - accuracy: 0.5057 - loss: 0.6932
Epoch 12/15
100/100

In [34]:
# Fine-tune the model
base_model.trainable = True  # Unfreeze all layers
fine_tune_at = 100  # Fine-tune from layer 100 onwards

# Freeze layers before the fine-tune layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model after making layers trainable
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model with a low learning rate
history_finetune = model.fit(train_generator, epochs=5, steps_per_epoch=100)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(valid_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 61s 242ms/step - accuracy: 0.5037 - loss: 1.0039 
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 128ms/step - accuracy: 0.4947 - loss: 0.8878
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 121ms/step - accuracy: 0.4967 - loss: 0.8014
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.5112 - loss: 0.7858
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 120ms/step - accuracy: 0.5145 - loss: 0.7432


NameError: name 'valid_generator' is not defined